In [28]:
# Scraper Météo Paris - Version avec ciblage précis basé sur l'inspection HTML

# Installation des dépendances nécessaires
# !pip install requests beautifulsoup4 pandas

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from IPython.display import display, HTML

# Constants
BASE_URL = "https://www.meteo-paris.com/ile-de-france/previsions"
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

def fetch_webpage(url):
    """Fetch the webpage content."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "text/html",
    }
    try:
        response = requests.get(url, headers=headers, timeout=30.0)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"Error fetching webpage: {e}")
        return ""

def extract_weather_data_targeted(html_content):
    """
    Extraction ciblée basée sur l'inspection HTML fournie.
    Cette méthode s'appuie sur les classes CSS spécifiques vues dans l'inspection.
    """
    if not html_content:
        return []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    weather_data = []
    
    print("=== Extraction ciblée basée sur l'inspection HTML ===")
    
    # 1. Chercher les jours de la semaine avec leur numéro
    # D'après l'inspection, ils sont dans des éléments avec des classes spécifiques
    day_pattern = re.compile(r'(Lun\.|Mar\.|Mer\.|Jeu\.|Ven\.|Sam\.|Dim\.)\s*(\d+)')
    
    # Chercher les éléments qui contiennent le jour et le numéro
    # D'après l'inspection, les jours sont dans des éléments avec des classes comme "flex-1" ou "text-xl"
    day_containers = []
    
    # Méthode 1: Chercher les éléments qui contiennent directement le texte du jour
    for text in soup.find_all(string=day_pattern):
        parent = text.parent
        # Remonter jusqu'à trouver un conteneur qui semble être une ligne de prévision
        container = parent
        while container and container.name != 'div' and not container.get('class'):
            container = container.parent
        
        if container and container not in day_containers:
            day_containers.append(container)
    
    # Méthode 2: Chercher par structure de classes vue dans l'inspection
    row_selectors = [
        'div.flex-1',
        'div[class*="flex-1"]',
        'div.relative.flex',
        'div[class*="relative"]',
        'div.bg-secondary-darken'
    ]
    
    for selector in row_selectors:
        elements = soup.select(selector)
        for elem in elements:
            text = elem.get_text()
            if day_pattern.search(text) and elem not in day_containers:
                day_containers.append(elem)
    
    print(f"Trouvé {len(day_containers)} conteneurs potentiels de jours")
    
    # Pour chaque conteneur de jour, chercher les températures associées
    for i, container in enumerate(day_containers):
        container_text = container.get_text().strip()
        day_match = day_pattern.search(container_text)
        
        if day_match:
            full_day = day_match.group(0)
            full_day = full_day.split('.')[0] + '.' + full_day.split('.')[1][:2]
             
            print(f"\nAnalyse du conteneur #{i+1} pour {full_day}")
            
            # D'après l'inspection, les températures sont dans des éléments circulaires
            # avec des classes comme "rounded-full" ou des couleurs de fond spécifiques
            
            # Méthode 1: Chercher des éléments avec la classe rounded-full
            temp_elements = container.select('.rounded-full') or container.select('[class*="rounded"]')
            
            # Méthode 2: Chercher des éléments avec les couleurs de fond vues dans l'inspection
            if not temp_elements:
                temp_elements = []
                
                # Chercher par la structure de classe vue dans l'inspection
                for elem in container.find_all():
                    elem_classes = elem.get('class', [])
                    elem_class_str = ' '.join(elem_classes) if elem_classes else ''
                    
                    # Ces éléments correspondent souvent aux températures
                    if (('flex' in elem_class_str and 'items-center' in elem_class_str) or
                        ('bg-' in elem_class_str) or
                        ('rounded' in elem_class_str)):
                        
                        elem_text = elem.get_text().strip()
                        # Vérifier que c'est un nombre simple
                        if re.match(r'^\d+$', elem_text):
                            temp_elements.append(elem)
            
            # Si nous avons trouvé des éléments de température
            if temp_elements and len(temp_elements) >= 2:
                temp_texts = [elem.get_text().strip() for elem in temp_elements]
                print(f"  Textes des éléments de température: {temp_texts}")
                
                # Filtrer pour ne garder que les éléments qui contiennent des nombres
                temp_elements = [elem for elem in temp_elements if re.match(r'^\d+$', elem.get_text().strip())]
                
                if len(temp_elements) >= 2:
                    # Les deux premiers éléments sont probablement min et max
                    temp_min = temp_elements[0].get_text().strip()
                    temp_max = temp_elements[1].get_text().strip()
                    
                    print(f"  Températures trouvées: min={temp_min}, max={temp_max}")
                    
                    weather_data.append({
                        "date": full_day,
                        "temperature_min": temp_min,
                        "temperature_max": temp_max,
                        "source": "Ciblage précis"
                    })
                else:
                    print(f"  Pas assez d'éléments de température trouvés après filtrage")
            else:
                print(f"  Pas assez d'éléments de température trouvés")
                
                # Dernière tentative: chercher des nombres simples dans le conteneur
                numbers = re.findall(r'\b\d{1,2}\b', container_text)
                
                if len(numbers) >= 3:  # Jour + min + max
                    # Le premier nombre est souvent le jour
                    if numbers[0] in full_day:
                        temp_min, temp_max = numbers[1], numbers[2]
                    else:
                        temp_min, temp_max = numbers[0], numbers[1]
                    
                    print(f"  Températures extraites du texte: min={temp_min}, max={temp_max}")
                    
                    weather_data.append({
                        "date": full_day,
                        "temperature_min": temp_min,
                        "temperature_max": temp_max,
                        "source": "Extraction de texte"
                    })
    
    # Si nous n'avons pas trouvé de données, essayer une approche plus générique
    if not weather_data:
        weather_data = extract_weather_data_generic(soup)
    
    return weather_data

def extract_weather_data_generic(soup):
    """
    Méthode d'extraction générique qui tente de trouver les données
    sans s'appuyer sur des classes spécifiques.
    """
    print("\n=== Extraction générique ===")
    weather_data = []
    
    # Pattern pour les jours de la semaine
    day_pattern = re.compile(r'(Lun\.|Mar\.|Mer\.|Jeu\.|Ven\.|Sam\.|Dim\.)\s*(\d+)')
    
    # Chercher tous les éléments qui pourraient contenir un jour de la semaine
    day_elements = []
    
    for element in soup.find_all(string=day_pattern):
        parent = element.parent
        day_elements.append(parent)
    
    print(f"Trouvé {len(day_elements)} éléments contenant des jours")
    
    # Pour chaque jour trouvé
    for day_elem in day_elements:
        day_text = day_elem.get_text().strip()
        day_match = day_pattern.search(day_text)
        
        if day_match:
            full_day = day_match.group(0)
            print(f"Analyse pour {full_day}")
            
            # Chercher les températures à proximité de cet élément
            # 1. Chercher dans les frères de l'élément
            siblings = list(day_elem.parent.find_all(recursive=False))
            
            # Chercher des éléments qui contiennent un seul nombre
            number_elements = []
            
            for sib in siblings:
                sib_text = sib.get_text().strip()
                if re.match(r'^\d+$', sib_text):
                    number_elements.append(sib)
            
            # Si nous avons trouvé au moins deux nombres
            if len(number_elements) >= 2:
                temp_min = number_elements[0].get_text().strip()
                temp_max = number_elements[1].get_text().strip()
                
                print(f"  Températures trouvées dans les frères: min={temp_min}, max={temp_max}")
                
                weather_data.append({
                    "date": full_day,
                    "temperature_min": temp_min,
                    "temperature_max": temp_max,
                    "source": "Éléments frères"
                })
                continue
            
            # 2. Si nous n'avons pas trouvé dans les frères directs, chercher dans le parent
            parent_container = day_elem.parent
            
            # Chercher des éléments qui contiennent un seul nombre
            number_elements = []
            
            for elem in parent_container.find_all():
                elem_text = elem.get_text().strip()
                if re.match(r'^\d+$', elem_text):
                    number_elements.append(elem)
            
            # Si nous avons trouvé au moins deux nombres
            if len(number_elements) >= 2:
                temp_min = number_elements[0].get_text().strip()
                temp_max = number_elements[1].get_text().strip()
                
                print(f"  Températures trouvées dans le parent: min={temp_min}, max={temp_max}")
                
                weather_data.append({
                    "date": full_day,
                    "temperature_min": temp_min,
                    "temperature_max": temp_max,
                    "source": "Éléments parent"
                })
                continue
            
            # 3. Dernière tentative: chercher des nombres dans le texte autour du jour
            row_text = parent_container.get_text()
            numbers = re.findall(r'\b\d{1,2}\b', row_text)
            
            if len(numbers) >= 3:  # Jour + min + max
                # Vérifier si le premier nombre est le jour
                if numbers[0] in full_day:
                    temp_min, temp_max = numbers[1], numbers[2]
                else:
                    temp_min, temp_max = numbers[0], numbers[1]
                
                print(f"  Températures extraites du texte: min={temp_min}, max={temp_max}")
                
                weather_data.append({
                    "date": full_day,
                    "temperature_min": temp_min,
                    "temperature_max": temp_max,
                    "source": "Texte"
                })
    
    return weather_data

def find_weather_by_structure(html_content):
    """
    Cette méthode cherche spécifiquement les éléments arrondis jaunes et verts
    qui contiennent les températures, comme vu dans les captures d'écran.
    """
    if not html_content:
        return []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    weather_data = []
    
    print("\n=== Recherche par structure visuelle ===")
    
    # Chercher les lignes qui contiennent les jours et les températures
    # D'après les captures d'écran, chaque ligne contient un jour (Lun. 21, etc.)
    # et deux cercles colorés pour les températures min et max
    
    # 1. D'abord, chercher toutes les lignes potentielles
    day_pattern = re.compile(r'(Lun\.|Mar\.|Mer\.|Jeu\.|Ven\.|Sam\.|Dim\.)\s*(\d+)')
    
    # Chercher les éléments qui pourraient être des lignes de prévision
    # D'après l'inspection, cela pourrait être des div avec des classes comme bg-secondary-darken
    rows = soup.select('div.bg-secondary-darken') or soup.select('div[class*="flex-1"]') or soup.select('div.relative')
    
    print(f"Trouvé {len(rows)} lignes potentielles")
    
    # Pour chaque ligne potentielle
    for i, row in enumerate(rows[:10]):  # Limiter à 10 pour le débogage
        row_text = row.get_text().strip()
        day_match = day_pattern.search(row_text)
        
        if day_match:
            full_day = day_match.group(0)
          
            print(f"\nLigne #{i+1} contenant {full_day}")
            
            # Chercher spécifiquement les éléments arrondis qui contiennent les températures
            # D'après les captures d'écran, ils ont souvent des classes contenant "rounded"
            # et peuvent avoir des arrière-plans colorés
            temp_elements = row.select('[class*="rounded"]')
            
            # Filtrer pour ne garder que ceux qui contiennent un nombre simple
            temp_elements = [elem for elem in temp_elements if re.match(r'^\d+$', elem.get_text().strip())]
            
            if temp_elements:
                print(f"  Trouvé {len(temp_elements)} éléments de température potentiels")
                
                # Afficher les classes pour aider au débogage
                for j, elem in enumerate(temp_elements[:5]):
                    elem_classes = ' '.join(elem.get('class', []))
                    elem_text = elem.get_text().strip()
                    print(f"  #{j+1}: {elem_text} - Classes: {elem_classes}")
                
                # Si nous avons au moins deux éléments (min et max)
                if len(temp_elements) >= 2:
                    temp_min = temp_elements[0].get_text().strip()
                    temp_max = temp_elements[1].get_text().strip()
                    
                    print(f"  Températures extraites: min={temp_min}, max={temp_max}")
                    
                    weather_data.append({
                        "date": full_day,
                        "temperature_min": temp_min,
                        "temperature_max": temp_max,
                        "source": "Structure visuelle"
                    })
            else:
                print(f"  Pas d'éléments arrondis trouvés dans cette ligne")
    
    return weather_data

def main():
    """Fonction principale qui exécute l'analyse complète."""
    print("=== Scraper Météo Paris - Version Ciblage Précis ===")
    html_content = fetch_webpage(BASE_URL)
    
    if not html_content:
        print("Échec de la récupération de la page web")
        return None
    
    print(f"Page web récupérée ({len(html_content)} caractères)")
    
    # Essayer d'abord l'extraction ciblée basée sur l'inspection HTML
    weather_data = extract_weather_data_targeted(html_content)
    
    # Si cela échoue, essayer la recherche par structure visuelle
    if not weather_data:
        weather_data = find_weather_by_structure(html_content)
    
    # Si toutes les méthodes échouent, créer des données placeholder
     
    
    
    
    # Afficher les résultats formatés
    print("\n=== Résultats formatés ===")
    for day in weather_data:
        print(f"{day['date']}, température mini {day['temperature_min']}°C, maxi {day['temperature_max']}°C")
    
     
    
    

# Pour exécuter l'analyse
df_meteo = main()

=== Scraper Météo Paris - Version Ciblage Précis ===
Page web récupérée (355873 caractères)
=== Extraction ciblée basée sur l'inspection HTML ===
Trouvé 31 conteneurs potentiels de jours

Analyse du conteneur #1 pour Lun.21
  Textes des éléments de température: ['10', '18']
  Températures trouvées: min=10, max=18

Analyse du conteneur #2 pour Mar.22
  Textes des éléments de température: ['9', '19']
  Températures trouvées: min=9, max=19

Analyse du conteneur #3 pour Mer.23
  Textes des éléments de température: ['', '9', '15']
  Températures trouvées: min=9, max=15

Analyse du conteneur #4 pour Jeu.24
  Textes des éléments de température: ['', '10', '16']
  Températures trouvées: min=10, max=16

Analyse du conteneur #5 pour Ven.25
  Textes des éléments de température: ['', '8', '18']
  Températures trouvées: min=8, max=18

Analyse du conteneur #6 pour Sam.26
  Textes des éléments de température: ['', '11', '16']
  Températures trouvées: min=11, max=16

Analyse du conteneur #7 pour Dim.2